# extraction

In [77]:
import os as os

# get the current working directory
notebook_dir = os.getcwd()

# define paths to CISI dataset files
cisi_all_path = os.path.join(notebook_dir, 'CISI.ALL')
cisi_qry_path = os.path.join(notebook_dir, 'CISI.QRY')
cisi_rel_path = os.path.join(notebook_dir, 'CISI.REL')

In [78]:
# extraction function for the CISI.ALL file
def extraction(file_name_path):
    with open(file_name_path) as f:
        file_lines = f.readlines()

    documents = []
    i = 0
    while i < len(file_lines):
        if file_lines[i].startswith(".I"):
            document_id = int(file_lines[i][3:]) # used slicing here to differentiate. maybe other methods too similar to online solutions
            if file_lines[i+1].startswith(".T") and file_lines[i+3].startswith(".A"):
                title = file_lines[i+2][:].strip()
            else:
                title = file_lines[i+2].strip() + " " + file_lines[i+3].strip()
            if file_lines[i+3].startswith(".A"):
                author = file_lines[i+4][:].strip()
            else:
                author = file_lines[i+5][:].strip()
            abstract_start = i+6
            for t in range(i+6, len(file_lines)):
                if file_lines[t].startswith(".X"):
                    abstract_end = t
                    break
            else:
                abstract_end = len(file_lines)
            abstract = " ".join(file_lines[abstract_start:abstract_end]).strip().replace('\n', ' ')
            abstract = " ".join(abstract.split())
            documents.append({"document_id": document_id, "title": title, "author": author, "abstract": abstract})
            i = abstract_end
        else:
            i += 1

    return documents

alternative - the entire text from .I to .X can be input into the dictionary and skip separating it into "author, title AND abstract" - just use document ID and "info" containing everything else
currently looking into this but cba

In [79]:
documents = extraction(cisi_all_path)

In [ ]:
documents # output is cleared here because this prints out all 1400 documents 

In [84]:
# query extraction for the CISI.QRY file
def extraction_query(file_name_path):
    with open(file_name_path) as f:
        file_lines = f.readlines()

    queries = []
    i = 0
    while i < len(file_lines):
        if file_lines[i].startswith(".I"):
            query_id = int(file_lines[i][3:])
            if file_lines[i+1].startswith(".W"):
                query_text = file_lines[i+2].strip()
                for t in range(i+3, len(file_lines)):
                    if file_lines[t].startswith(".I"):
                        break
                    query_text += " " + file_lines[t].strip()
            else:
                query_text = ""
            queries.append({"query_id": query_id, "query_text": query_text})
        i += 1

    return queries

In [85]:
queries = extraction_query(cisi_qry_path)

In [ ]:
queries # again output is cleared

In [87]:
#to do list
#make sure author extraction works with multiple authors
#add rule to abstract where the .W is removed
#double check all abstracts and see if all the lines are actually included

In [87]:
#complete extraction for the CISI.REL file

# preprocessing

- tokenize all the words in the dictionary
- remove stopwords
- use stemming/don't use stemming (will this increase performance?)

# indexing

 - use elasticsearch to index documents and terms

# retrieval

- create bm25 model
- create vsm model
- design query parser/processor

# evaluation

- somehow use the cisi.qry file to compare our search engine to the relevance judgements (idk what this file is)
- obtain the outputs and write a function that calculates the precision, recall and f1 metrics
- write a simple function that takes these metrics and puts them into a bar chart

- also obtain the MAP for the cross-model comparison
- write a simple function that takes the MAP and puts it into a bar chart

1. which model performed the best
2. were there exceptions/unusual findings
3. conclusion/findings
4. FUTURE - OPTIMISATION
    - is there room for fine-tuning/optimisation
    - give details of this
    - implement it if there is time